# **Political Voting Survey Dashboard 🌏**

**Introduction**

This project aims to showcase my skills in Data Visualizations and Dashboard Development with Tableau using generated dummy data with an example case in political surveys. Also, while the Visualization using Tableau are using CSV Files, due to Tableau Public Limitations, I am also going to develop an ingestion process from Generated Data to PostgreSQL for further real-time (or updated) visualizations.

Due to current limitations in access data, I will generate dummy data using APIs from multiple random generators with a location in Indonesia. This project does not at all represent any political fact about any region and are artificial created for learning and skills showcasing purposes.

**Tables:**

- Votes (6000 Unique Values)
    - voter_name
    - voter_id
    - candidate_id
    - region_id
- Candidate (4 Unique Values)
    - Candidate_id
    - Nama (Buat 4)
    - Party_id (Buat 4)
- region (33 Unique Values)
    - region_id
    - nama_wilayah
- Electoral Vote ← Voters (dibuat dari yang menang masing-masing)
    - Wilayah (Ada 33)
    - Vote Count
    - Candidate_name
    - Party
- Party
    - party_id
    - party_name
    - member_count

In [ ]:
import pandas as pd
import random
from helper.data_generate import generate_person

**Region** 

In [ ]:
region_df = pd.read_csv('sources/regions_id.csv')
region_df = region_df.reset_index().rename(columns={"index": "id"})

region_df.rename(columns={"Provinsi di Indonesia":"province",
                          "Jumlah Penduduk Menurut Provinsi di Indonesia (Ribu Jiwa)":"population_count_in_thousands"},
                 inplace=True)

region_df = region_df[~region_df["province"].isin(["Indonesia"])]
region_df['id'] = region_df['id'] + 1

In [ ]:
region_df

**Party** 

In [ ]:
party_names = ["Partai Sosial Demokrasi Indonesia", "Partai Republik Nasionalis Nusantara", "Partai Amanat Indonesia", "Partai Kerjasama dan Utusan Rakyat"]
party_list = []
id = 1

for i in party_names:
    party = {}
    party["id"] = id
    id += 1
    party["party_name"] = i
    party["total_members"] = random.randint(1500, 5000)

    party_list.append(party)

In [ ]:
party_df = pd.DataFrame(party_list)

In [ ]:
party_df

**Candidate** 

In [ ]:
generate_person()

In [ ]:
candidates = []

for i in party_df["id"]:
    candidate = {}
    person = generate_person()

    candidate['id'] = i
    candidate['name'] = person['name']['first'] + ' ' + person['name']['last']
    candidate['gender'] = person['gender']
    candidate['party_id'] = i

    candidates.append(candidate)

candidates_df = pd.DataFrame(candidates) 

In [ ]:
candidates_df

**Voters** 

In [58]:
voters_csv_df = pd.read_csv("csv/voters.csv")
voters_csv_df.drop(columns=['Unnamed: 0'], inplace=True)

In [61]:
voters_csv_df

,name,gender,candidate_id,region_id
0,Mohamad Olivier,male,3,22
1,Ulrich Dumont,male,3,22
2,Catherine Petit,female,3,22
3,Lars Bourgeois,male,3,22
4,Kerstin Robert,female,3,22
...,...,...,...,...
11724,Maeva Leclerc,female,3,22
11725,Monique Blanc,female,3,22
11726,Eliana Robin,female,3,22
11727,Johan Durand,male,3,22


In [ ]:
voters = []

def generate_voter():
    person = generate_person()
    voter = {}

    voter['name'] = person['name']['first'] + ' ' + person['name']['last']
    voter['gender'] = person['gender']
    voter['candidate_id'] = random.choice(candidates_df['id'].tolist())
    voter['region_id'] = random.choice(region_df['id'].tolist())

    print(voter)
    return voter


In [ ]:
for x in range(7):
    for i in range(250):
        print(str(i) + " " + str(x))
        voter = generate_voter()
        voters.append(voter)

In [55]:
voters_df = pd.DataFrame(voters)

In [64]:
voters_csv_df = pd.concat([voters_csv_df, voters_df], ignore_index=True)
voters_csv_df = voters_csv_df.drop_duplicates()
voters_csv_df.rename(columns={
    'vote': 'candidate_id'
}, inplace=True)

voters_csv_df.to_csv("csv/voters.csv")

In [65]:
voters_csv_df['candidate_id'] = voters_csv_df.apply(lambda _: np.random.choice(candidates_df['id'].tolist()), axis=1)
voters_csv_df['region_id'] = voters_csv_df.apply(lambda _: np.random.choice(region_df['id'].tolist()), axis=1)

In [66]:
voters_csv_df

,name,gender,candidate_id,region_id
0,Mohamad Olivier,male,2,15
1,Ulrich Dumont,male,1,15
2,Catherine Petit,female,3,8
3,Lars Bourgeois,male,2,14
4,Kerstin Robert,female,2,6
...,...,...,...,...
11724,Maeva Leclerc,female,1,26
11725,Monique Blanc,female,2,12
11726,Eliana Robin,female,1,9
11727,Johan Durand,male,1,26


**Electoral** 

In [74]:
votes_with_candidates = pd.merge(voters_csv_df, candidates_df[['id', 'name', 'party_id']], left_on='candidate_id', right_on='id', how='inner')

votes_with_region = pd.merge(votes_with_candidates, region_df[['id', 'province']], left_on='region_id', right_on='id', how='inner')

votes_cleansed = pd.merge(votes_with_region, party_df[['id','party_name']], left_on='party_id', right_on='id', how='inner')

votes_cleansed.drop(columns=['id_x','id_y','id'], inplace=True)

votes_cleansed.rename(columns={'name_x': 'voter_name', 'name_y':'candidate_name', 'vote':'candidate_id'}, inplace=True)

votes_cleansed['vote_values'] = 1

votes_cleansed

,voter_name,gender,candidate_id,region_id,candidate_name,party_id,province,party_name,vote_values
0,Mohamad Olivier,male,2,15,George Dubois,2,DI Yogyakarta,Partai Republik Nasionalis Nusantara,1
1,Ulrich Dumont,male,1,15,Dominic Lefevre,1,DI Yogyakarta,Partai Sosial Demokrasi Indonesia,1
2,Catherine Petit,female,3,8,Nelli Aubert,3,Kep. Bangka Belitung,Partai Amanat Indonesia,1
3,Lars Bourgeois,male,2,14,George Dubois,2,Jawa Tengah,Partai Republik Nasionalis Nusantara,1
4,Kerstin Robert,female,2,6,George Dubois,2,Jambi,Partai Republik Nasionalis Nusantara,1
...,...,...,...,...,...,...,...,...,...
11714,Maeva Leclerc,female,1,26,Dominic Lefevre,1,Sulawesi Barat,Partai Sosial Demokrasi Indonesia,1
11715,Monique Blanc,female,2,12,George Dubois,2,Jawa Barat,Partai Republik Nasionalis Nusantara,1
11716,Eliana Robin,female,1,9,Dominic Lefevre,1,Bengkulu,Partai Sosial Demokrasi Indonesia,1
11717,Johan Durand,male,1,26,Dominic Lefevre,1,Sulawesi Barat,Partai Sosial Demokrasi Indonesia,1


In [72]:
votes_cleansed.groupby('province').agg({
    'voter_name':'count'
})

,voter_name
province,
Aceh,335
Bali,383
Banten,360
Bengkulu,373
DI Yogyakarta,335
DKI Jakarta,335
Gorontalo,354
Jambi,339
Jawa Barat,348


In [73]:
votes_cleansed[votes_cleansed['province'] == 'Aceh']

,voter_name,gender,candidate_id,region_id,candidate_name,party_id,province,party_name
31,Timo Nguyen,male,1,1,Dominic Lefevre,1,Aceh,Partai Sosial Demokrasi Indonesia
32,Niels Robert,male,4,1,Emily Meyer,4,Aceh,Partai Kerjasama dan Utusan Rakyat
59,Louisa Pierre,female,3,1,Nelli Aubert,3,Aceh,Partai Amanat Indonesia
72,Thomas Lemoine,male,2,1,George Dubois,2,Aceh,Partai Republik Nasionalis Nusantara
73,Markus Gerard,male,1,1,Dominic Lefevre,1,Aceh,Partai Sosial Demokrasi Indonesia
...,...,...,...,...,...,...,...,...
11673,Beat David,male,4,1,Emily Meyer,4,Aceh,Partai Kerjasama dan Utusan Rakyat
11698,Lia Garnier,female,3,1,Nelli Aubert,3,Aceh,Partai Amanat Indonesia
11700,Ida Nguyen,female,3,1,Nelli Aubert,3,Aceh,Partai Amanat Indonesia
11707,Léon Fleury,male,2,1,George Dubois,2,Aceh,Partai Republik Nasionalis Nusantara


In [68]:
# Result total pemenang
votes_cleansed.groupby('candidate_name').agg({
    'voter_name':'count'
})

,voter_name
candidate_name,
Dominic Lefevre,2851
Emily Meyer,2927
George Dubois,2969
Nelli Aubert,2972


In [75]:
votes_cleansed.to_csv('csv/votes_cleansed.csv')

In [69]:
# Data detail untuk masing-masing provinsi
details_province = votes_cleansed.groupby(['province', 'candidate_name']).agg({
    'voter_name': ['count']
})

details_province.rename(columns={'voter_name': 'vote_count'}, inplace=True)
details_province.columns = details_province.columns.droplevel(1)
details_province.reset_index()

details_province

vote_count
province         candidate_name             
Aceh             Dominic Lefevre          85
                 Emily Meyer              86
                 George Dubois            89
                 Nelli Aubert             75
Bali             Dominic Lefevre          91
...                                      ...
Sumatera Selatan Nelli Aubert             88
Sumatera Utara   Dominic Lefevre          69
                 Emily Meyer              73
                 George Dubois            92
                 Nelli Aubert             85

[136 rows x 1 columns]

In [70]:
# Data pememenang setiap provinsi
winners = details_province.groupby('province')['vote_count'].idxmax()
winning_candidates = details_province.loc[winners].reset_index()
winning_candidates.to_csv('csv/regional_results.csv')

winning_candidates

,province,candidate_name,vote_count
0,Aceh,George Dubois,89
1,Bali,Nelli Aubert,110
2,Banten,Dominic Lefevre,103
3,Bengkulu,George Dubois,101
4,DI Yogyakarta,Emily Meyer,97
5,DKI Jakarta,Emily Meyer,99
6,Gorontalo,Emily Meyer,111
7,Jambi,Dominic Lefevre,102
8,Jawa Barat,Nelli Aubert,95
9,Jawa Tengah,Nelli Aubert,97


In [71]:
# Result total pemenang
winning_candidates.groupby('candidate_name').agg({
    'province':'count'
})

,province
candidate_name,
Dominic Lefevre,5
Emily Meyer,9
George Dubois,10
Nelli Aubert,10
